In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import os

In [14]:
def merge_dataframes(df1, df2, column_patterns):
    """
    Merge the first 13 columns of two DataFrames.

    Parameters:
    df1 (DataFrame): First DataFrame.
    df2 (DataFrame): Second DataFrame.

    Returns:
    DataFrame: Merged DataFrame containing the first 13 columns of each DataFrame.
    """
    # Determine the number of columns to merge (minimum between the two DataFrames)
    num_columns_to_merge = max(min(df1.shape[1], df2.shape[1]), len(column_patterns))
    # Select the first 13 columns from each DataFrame
    df1_subset = df1.iloc[:, :num_columns_to_merge]
    df2_subset = df2.iloc[:, :num_columns_to_merge]

    # Merge the subsets along columns
    merged_df = pd.concat([df1_subset, df2_subset], axis=0)

    return merged_df

def merge_dataframes_with_previous_files(file_paths, column_patterns):
    """
    Merge the first 13 columns of DataFrames from multiple Excel files with two sheets each.
    The column names from the first DataFrame are kept.

    Parameters:
    file_paths (list): List of file paths to Excel files.

    Returns:
    DataFrame: Merged DataFrame containing the first 13 columns of each DataFrame from all files.
    """
    merged_df = None
    merged_df_temp = None
    
    # Iterate over each file path
    for file_path in file_paths:
        print(f"\nLoading file : {file_path}")
        # Load Excel file into separate DataFrames
        # Fetch only first sheet as it is RES 
        df = pd.read_excel(file_path, header=1, sheet_name=[0], parse_dates=True)
        print(f"Number of sheets detected: {len(df.keys())}")

        # Merge in sheet level
        for sheet_name, df_sheet in df.items():
            # Filter according to pattern columns to keep
            df_sheet = df_sheet.filter(regex='|'.join(column_patterns))
            df_sheet['file'] = file_path
            print(f"Number of columns detected in {sheet_name} : {len(df_sheet.columns)}")
            if merged_df_temp is None:
                # Merge the Sheet DataFrames
                merged_df_temp = df_sheet
            else:
                merged_df_temp = merge_dataframes(merged_df_temp, df_sheet, column_patterns)

        # Merge in file level
        # If merged_df is None, assign it the merged DataFrame, else merge with the previous merged DataFrame
        if merged_df is None:
            merged_df = merged_df_temp
        else:
            merged_df = merge_dataframes(merged_df, merged_df_temp, column_patterns)

    return merged_df

### Special Projects Fetching

In [15]:
# Define the directory containing Excel files
directory = "../../data/special_projects/"

# Get all Excel files in the directory
file_paths = [os.path.join(directory, file) for file in os.listdir(directory) if file.endswith(".xlsx")]
column_patterns = ['ΑΙΤΗΣΗ', 'ΜΗΤΡΩΟ', 'ΗΜΕΡΟΜΗΝΙΑ ΥΠΟΒΟΛΗΣ.*', 'ΕΤΑΙΡΕΙΑ', 'ΗΜΕΡ.*ΕΚΔ.*ΠΑΡΑΓ.*', 'ΗΜΕΡ.*ΛΗΞΗ.*ΠΑΡΑΓ.*', 'ΠΕΡΙΦΕΡΕΙΑ', 'ΠΕΡΙΦΕΡΕΙΑΚΗ ΕΝΟΤΗΤΑ', 'ΔΗΜΟ.* ', 'ΔΗΜΟΤΙΚΗ ΕΝΟΤΗΤΑ', 'ΘΕΣΗ', '.*(MW)', 'ΤΕΧΝΟΛ.*'] 

# Sort file paths based on their names to process them in order
file_paths.sort()



In [16]:
# Merge DataFrames from all Excel files
merged_df = merge_dataframes_with_previous_files(file_paths, column_patterns)


Loading file : ../../data/special_projects/202012.xlsx
Number of sheets detected: 1
Number of columns detected in 0 : 14

Loading file : ../../data/special_projects/202103.xlsx
Number of sheets detected: 1
Number of columns detected in 0 : 14

Loading file : ../../data/special_projects/202105.xlsx
Number of sheets detected: 1
Number of columns detected in 0 : 14

Loading file : ../../data/special_projects/202108.xlsx
Number of sheets detected: 1
Number of columns detected in 0 : 14

Loading file : ../../data/special_projects/202110.xlsx
Number of sheets detected: 1
Number of columns detected in 0 : 14

Loading file : ../../data/special_projects/202203.xlsx
Number of sheets detected: 1
Number of columns detected in 0 : 14

Loading file : ../../data/special_projects/202209.xlsx
Number of sheets detected: 1
Number of columns detected in 0 : 14

Loading file : ../../data/special_projects/202212.xlsx
Number of sheets detected: 1
Number of columns detected in 0 : 14

Loading file : ../../da

/tmp/ipykernel_5569/310484181.py:49: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_sheet['file'] = file_path


Number of sheets detected: 1
Number of columns detected in 0 : 14

Loading file : ../../data/special_projects/202303.xlsx


/tmp/ipykernel_5569/310484181.py:49: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_sheet['file'] = file_path


Number of sheets detected: 1
Number of columns detected in 0 : 14

Loading file : ../../data/special_projects/202305.xlsx


...

Number of sheets detected: 1
Number of columns detected in 0 : 14

Loading file : ../../data/special_projects/202307.xlsx


/tmp/ipykernel_5569/310484181.py:49: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_sheet['file'] = file_path


Number of sheets detected: 1
Number of columns detected in 0 : 14

Loading file : ../../data/special_projects/202309.xlsx


/tmp/ipykernel_5569/310484181.py:49: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_sheet['file'] = file_path


Number of sheets detected: 1
Number of columns detected in 0 : 14

Loading file : ../../data/special_projects/202310.xlsx
Number of sheets detected: 1
Number of columns detected in 0 : 14

Loading file : ../../data/special_projects/202311.xlsx
Number of sheets detected: 1
Number of columns detected in 0 : 14

Loading file : ../../data/special_projects/202402.xlsx
Number of sheets detected: 1
Number of columns detected in 0 : 14

Loading file : ../../data/special_projects/202403.xlsx
Number of sheets detected: 1
Number of columns detected in 0 : 14


In [17]:
merged_df

,ΑΙΤΗΣΗ,ΗΜΕΡΟΜΗΝΙΑ ΥΠΟΒΟΛΗΣ ΑΙΤΗΣΗΣ,ΕΤΑΙΡΕΙΑ,ΑΡ. ΜΗΤΡΩΟΥ ΑΔΕΙΩΝ ΡΑΕ,ΗΜΕΡΟΜΗΝΙΑ ΕΚΔ. ΑΔ.ΠΑΡΑΓΩΓΗΣ,ΗΜΕΡΟΜΗΝΙΑ ΛΗΞΗΣ ΑΔ.ΠΑΡΑΓΩΓΗΣ,ΠΕΡΙΦΕΡΕΙΑ,ΠΕΡΙΦΕΡΕΙΑΚΗ ΕΝΟΤΗΤΑ,ΔΗΜΟΣ,ΔΗΜΟΤΙΚΗ ΕΝΟΤΗΤΑ,ΘΕΣΗ,ΙΣΧΥΣ (MW),ΤΕΧΝΟΛΟΓΙΑ,file
0,B-00279,2001-02-19,ΔΕΗ ΑΝΑΝΕΩΣΙΜΕΣ ΑΕ,ΑΔ-00488,2003-03-04,2028-03-04 00:00:00,ΒΟΡΕΙΟΥ ΑΙΓΑΙΟΥ,ΛΕΣΒΟΥ,ΛΕΣΒΟΥ,ΠΕΤΡΑΣ,ΣΤΥΨΗ/ΠΕΡΙΟΧΗ ΑΡΓΕΝΟΥ,8.000,ΓΕΩΘΕΡΜΙΑ,../../data/special_projects/202012.xlsx
1,Γ-00798,2004-11-18,ΤΕΡΝΑ ΕΝΕΡΓΕΙΑΚΗ ΑΙ ΓΙΩΡΓΗΣ ΑΕ,ΑΔ-00911,2006-03-23,2021-03-23 00:00:00,ΑΤΤΙΚΗΣ,ΝΗΣΩΝ,ΥΔΡΑΣ,ΥΔΡΑΣ,ΝΗΣΙΔΑ ΑΓΙΟΣ ΓΕΩΡΓΙΟΣ,45.000,ΑΙΟΛΙΚΑ,../../data/special_projects/202012.xlsx
2,Γ-01329,2006-01-24,ΑΙΟΛΙΚΑ ΠΑΡΚΑ ΠΑΛΙΟΠΥΡΓΟΣ ΑΕ,ΑΔ-00981,2006-10-02,2021-10-02 00:00:00,ΣΤΕΡΕΑΣ ΕΛΛΑΔΟΣ,ΕΥΒΟΙΑΣ,ΚΑΡΥΣΤΟΥ,ΚΑΡΥΣΤΟΥ & ΚΑΦΗΡΕΩΣ,ΠΑΛΙΟΠΥΡΓΟΣ,13.800,ΑΙΟΛΙΚΑ,../../data/special_projects/202012.xlsx
3,Γ-01089,2005-07-22,ΤΕΡΝΑ ΕΝΕΡΓΕΙΑΚΗ ΑΙ ΓΙΩΡΓΗΣ ΑΕ,ΑΔ-00989,2006-10-16,2021-10-16 00:00:00,ΑΤΤΙΚΗΣ,ΝΗΣΩΝ,ΥΔΡΑΣ,ΥΔΡΑΣ,ΝΗΣΙΔΑ ΑΓΙΟΣ ΓΕΩΡΓΙΟΣ,24.000,ΑΙΟΛΙΚΑ,../../data/special_projects/202012.xlsx
4,Γ-01328,2006-01-24,ΑΙΟΛΙΚΑ ΠΑΡΚΑ ΠΛΑΤΑΝΟΣ ΜΟΝΟΠΡΟΣΩΠΗ ΑΕ,ΑΔ-01257,2009-06-16,2034-06-16 00:00:00,ΣΤΕΡΕΑΣ ΕΛΛΑΔΟΣ,ΕΥΒΟΙΑΣ,ΚΑΡΥΣΤΟΥ,ΜΑΡΜΑΡΙΟΥ,ΠΛΑΤΑΝΟΣ,13.800,ΑΙΟΛΙΚΑ,../../data/special_projects/202012.xlsx
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
193,Γ-011373_Α,2020-12-16,ΑΙΟΛΙΚΗ ΘΕΟΔΩΡΩΝ ΑΝΩΝΥΜΗ ΒΙΟΜΗΧΑΝΙΚΗ ΚΑΙ ΕΝΕΡ...,ΑΔ-04888,2022-11-04,2047-11-04 00:00:00,ΠΕΛΟΠΟΝΝΗΣΟΥ,ΑΡΚΑΔΙΑΣ,"ΜΕΓΑΛΟΠΟΛΗΣ,ΓΟΡΤΥΝΙΑΣ,ΤΡΙΠΟΛΗΣ","ΜΕΓΑΛΟΠΟΛΗΣ,ΤΡΙΚΟΛΩΝΩΝ,ΦΑΛΑΝΘΟΥ","ΜΠΛΕΣΙΒΟΣ, ΨΑΡΟΒΟΥΝΙ, ΛΙΜΝΕΣ ΡΟΔΙΑΣ, ΔΙΑΚΟΠΙ, ...",187.110,ΑΙΟΛΙΚΑ,../../data/special_projects/202403.xlsx
194,Γ-011380_Α,2020-12-16,ΑΙΟΛΙΚΗ ΘΕΟΔΩΡΩΝ ΑΝΩΝΥΜΗ ΒΙΟΜΗΧΑΝΙΚΗ ΚΑΙ ΕΝΕΡ...,ΑΔ-04889,2022-11-04,2047-11-04 00:00:00,"ΔΥΤΙΚΗΣ ΕΛΛΑΔΑΣ,ΠΕΛΟΠΟΝΝΗΣΟΥ","ΗΛΕΙΑΣ,ΑΡΚΑΔΙΑΣ,ΜΕΣΣΗΝΙΑΣ","ΑΝΔΡΙΤΣΑΙΝΑΣ - ΚΡΕΣΤΕΝΩΝ,ΜΕΓΑΛΟΠΟΛΗΣ,ΟΙΧΑΛΙΑΣ,...","ΑΛΙΦΕΙΡΑΣ,ΑΝΔΡΙΤΣΑΙΝΗΣ,ΓΟΡΤΥΝΟΣ,ΕΙΡΑΣ,ΖΑΧΑΡΩΣ,...","ΓΥΜΝΟΡΡΑΧΗ, ΡΟΣΚΙΟ, ΜΠΡΙΝΙΑ, ΜΠΑΝΤΑΒΑΣ, ΚΟΥΚΟΥ...",417.600,ΑΙΟΛΙΚΑ,../../data/special_projects/202403.xlsx
195,Γ-011383_Α,2020-12-16,ΑΙΟΛΙΚΗ ΘΕΟΔΩΡΩΝ ΑΝΩΝΥΜΗ ΒΙΟΜΗΧΑΝΙΚΗ ΚΑΙ ΕΝΕΡ...,ΑΔ-04890,2022-11-04,2047-11-04 00:00:00,"ΠΕΛΟΠΟΝΝΗΣΟΥ,ΔΥΤΙΚΗΣ ΕΛΛΑΔΑΣ","ΑΡΚΑΔΙΑΣ,ΑΧΑΪΑΣ,ΚΟΡΙΝΘΙΑΣ","ΓΟΡΤΥΝΙΑΣ,ΚΑΛΑΒΡΥΤΩΝ,ΤΡΙΠΟΛΗΣ,ΕΡΥΜΑΝΘΟΥ,ΣΙΚΥΩΝΙΩΝ","ΒΥΤΙΝΑΣ,ΚΑΛΑΒΡΥΤΩΝ,ΚΛΕΙΤΟΡΙΑΣ, ΚΛΕΙΤΟΡΟΣ,ΚΟΝ...","ΛΑΚΩΜΑΤΑ, ΠΑΝΑΓΙΑ, ΣΑΙΤΑΣ, ΔΡΑΚΟΒΟΥΝΙ, ΒΥΘΟΥΛΑ...",280.665,ΑΙΟΛΙΚΑ,../../data/special_projects/202403.xlsx
196,Γ-013272,2021-06-09,VOLTON ΕΛΛΗΝΙΚΗ ΕΝΕΡΓΕΙΑΚΗ ΑΝΩΝΥΜΗ ΕΤΑΙΡΕΙΑ,ΑΔ-04891,2022-11-04,2047-11-04 00:00:00,ΠΕΛΟΠΟΝΝΗΣΟΥ,"ΑΡΓΟΛΙΔΑΣ, ΑΡΚΑΔΙΑΣ","ΑΡΓΟΥΣ -ΜΥΚΗΝΩΝ, ΤΡΙΠΟΛΗΣ, ΒΟΡΕΙΑΣ ΚΥΝΟΥΡΙΑΣ, ...","ΑΡΓΟΥΣ, ΒΑΛΤΕΤΣΙΟΥ, ΒΟΡΕΙΑΣ ΚΥΝΟΥΡΙΑΣ, ΛΥΡΚΕΙΑ...",ΑΓΙΟΣ ΠΕΤΡΟΣ-ΒΑΛΤΕΤΣΙ-ΝΕΣΤΑΝΗ-ΤΣΕΜΠΕΡΟΥ,298.200,ΑΙΟΛΙΚΑ,../../data/special_projects/202403.xlsx


In [18]:
def convert_to_datetime(df):
    """
    Convert date-like columns in a DataFrame to datetime type.

    Parameters:
    df (DataFrame): Input DataFrame.

    Returns:
    DataFrame: DataFrame with date-like columns converted to datetime type.
    """
    # Copy the DataFrame to avoid modifying the original DataFrame
    df_copy = df.copy()
    
    # Iterate through each column
    for col in df_copy.columns:
        print(f"Attempting column: {col}")
        # Check if the column can be converted to datetime
        try:
            df_copy[col] = pd.to_datetime(df_copy[col], format="%Y-%m-%d")
            print(f"Column {col} transformed succesfully.\n")
        except:
            print(f"Column {col} is incompatible with datetime.\n")
            # If conversion fails, skip to the next column
            continue
    return df_copy

In [19]:
def assign_naming(df, colnames=None):

    if colnames is None:
        colnames = ['ΗΜΕΡΟΜΗΝΙΑ ΥΠΟΒΟΛΗΣ ΑΙΤΗΣΗΣ', 'ΕΤΑΙΡΕΙΑ', 'ΗΜΕΡΟΜΗΝΙΑ ΕΚΔ. ΑΔ.ΠΑΡΑΓΩΓΗΣ',
        'ΗΜΕΡΟΜΗΝΙΑ ΛΗΞΗΣ ΑΔ.ΠΑΡΑΓΩΓΗΣ', 'ΠΕΡΙΦΕΡΕΙΑ', 'ΠΕΡΙΦΕΡΕΙΑΚΗ ΕΝΟΤΗΤΑ',
        'ΔΗΜΟ.* ', 'ΔΗΜΟΤΙΚΗ ΕΝΟΤΗΤΑ', 'ΘΕΣΗ', '(MW)', 'ΤΕΧΝΟΛΟΓΙΑ']

    num_columns_to_merge = max(len(df.columns), 13)

    df = df.set_axis(colnames[:num_columns_to_merge], axis='columns')

    return df

In [20]:
# Define pairs of columns to be merged
# merge_pairs = {
#     'ΗΜΕΡΟΜΗΝΙΑ ΥΠΟΒΟΛΗΣ ΑΙΤΗΣΗΣ': 'ΗΜΕΡΟΜΗΝΙΑ ΥΠΟΒΟΛΗΣ',
#     'ΗΜΕΡΟΜΗΝΙΑ ΕΚΔ. ΑΔ.ΠΑΡΑΓΩΓΗΣ': 'ΗΜΕΡ. ΕΚΔΟΣΗΣ ΒΕΒΑΙΩΣΗΣ ΠΑΡΑΓΩΓΟΥ',
#     # 'ΗΜΕΡΟΜΗΝΙΑ ΛΗΞΗΣ ΑΔ.ΠΑΡΑΓΩΓΗΣ': 'ΗΜΕΡ. ΛΗΞΗΣ ΒΕΒΑΙΩΣΗΣ ΠΑΡΑΓΩΓΟΥ',
#     # 'ΗΜΕΡΟΜΗΝΙΑ ΕΚΔ. ΑΔ.ΠΑΡΑΓΩΓΗΣ': 'ΗΜΕΡ. ΕΚΔ. ΑΔ.ΠΑΡΑΓΩΓΗΣ',
#     # 'ΙΣΧΥΣ (MW)': 'Ισχύς (MW)',
#     # 'ΗΜΕΡΟΜΗΝΙΑ ΕΚΔ. ΑΔ.ΠΑΡΑΓΩΓΗΣ' : 'ΗΜΕΡ. ΕΚΔΟΣΗΣ ΒΕΒΑΙΩΣΗΣ ΠΑΡΑΓΩΓΟΥ',
#     # 'ΤΕΧΝΟΛΟΓΙΑ' : 'ΤΕΧΝΟΛΟΓΙΑ ΑΠΕ',
#     # 'ΗΜΕΡΟΜΗΝΙΑ ΕΚΔ. ΑΔ.ΠΑΡΑΓΩΓΗΣ' : 'ΗΜΕΡ. ΕΚΔ. ΑΔ.ΠΑΡΑΓΩΓΗΣ',
#     # 'ΗΜΕΡΟΜΗΝΙΑ ΛΗΞΗΣ ΑΔ.ΠΑΡΑΓΩΓΗΣ' : 'ΗΜΕΡ. ΛΗΞΗΣ ΑΔ.ΠΑΡΑΓΩΓΗΣ'
# }

def merge_columns(df, merge_pairs):
    for col_to_assign, col_to_del in merge_pairs.items():
        # Filter rows where values exist in the column to assign
        rows_with_values = df[col_to_del].notna()
        # Fill NaN values in the column to assign with values from the column to delete only where values exist
        df.loc[rows_with_values, col_to_assign] = df.loc[rows_with_values, col_to_del]
        # Drop the column to delete
        df.drop(columns=col_to_del, inplace=True)
    return df


df_merged_test = convert_to_datetime(merged_df.copy())
# Merge columns based on merge pairs
# df_merged_test = merge_columns(df_merged_test.copy(), merge_pairs)

Attempting column: ΑΙΤΗΣΗ
Column ΑΙΤΗΣΗ is incompatible with datetime.

Attempting column: ΗΜΕΡΟΜΗΝΙΑ ΥΠΟΒΟΛΗΣ ΑΙΤΗΣΗΣ
Column ΗΜΕΡΟΜΗΝΙΑ ΥΠΟΒΟΛΗΣ ΑΙΤΗΣΗΣ transformed succesfully.

Attempting column: ΕΤΑΙΡΕΙΑ
Column ΕΤΑΙΡΕΙΑ is incompatible with datetime.

Attempting column: ΑΡ. ΜΗΤΡΩΟΥ ΑΔΕΙΩΝ ΡΑΕ
Column ΑΡ. ΜΗΤΡΩΟΥ ΑΔΕΙΩΝ ΡΑΕ is incompatible with datetime.

Attempting column: ΗΜΕΡΟΜΗΝΙΑ ΕΚΔ. ΑΔ.ΠΑΡΑΓΩΓΗΣ
Column ΗΜΕΡΟΜΗΝΙΑ ΕΚΔ. ΑΔ.ΠΑΡΑΓΩΓΗΣ transformed succesfully.

Attempting column: ΗΜΕΡΟΜΗΝΙΑ ΛΗΞΗΣ ΑΔ.ΠΑΡΑΓΩΓΗΣ
Column ΗΜΕΡΟΜΗΝΙΑ ΛΗΞΗΣ ΑΔ.ΠΑΡΑΓΩΓΗΣ is incompatible with datetime.

Attempting column: ΠΕΡΙΦΕΡΕΙΑ
Column ΠΕΡΙΦΕΡΕΙΑ is incompatible with datetime.

Attempting column: ΠΕΡΙΦΕΡΕΙΑΚΗ ΕΝΟΤΗΤΑ
Column ΠΕΡΙΦΕΡΕΙΑΚΗ ΕΝΟΤΗΤΑ is incompatible with datetime.

Attempting column: ΔΗΜΟΣ 
Column ΔΗΜΟΣ  is incompatible with datetime.

Attempting column: ΔΗΜΟΤΙΚΗ ΕΝΟΤΗΤΑ
Column ΔΗΜΟΤΙΚΗ ΕΝΟΤΗΤΑ is incompatible with datetime.

Attempting column: ΘΕΣΗ
Column ΘΕΣΗ is incompatible with datetime.



In [21]:
df_merged_test[df_merged_test['ΑΙΤΗΣΗ'] == 'B-00279']

,ΑΙΤΗΣΗ,ΗΜΕΡΟΜΗΝΙΑ ΥΠΟΒΟΛΗΣ ΑΙΤΗΣΗΣ,ΕΤΑΙΡΕΙΑ,ΑΡ. ΜΗΤΡΩΟΥ ΑΔΕΙΩΝ ΡΑΕ,ΗΜΕΡΟΜΗΝΙΑ ΕΚΔ. ΑΔ.ΠΑΡΑΓΩΓΗΣ,ΗΜΕΡΟΜΗΝΙΑ ΛΗΞΗΣ ΑΔ.ΠΑΡΑΓΩΓΗΣ,ΠΕΡΙΦΕΡΕΙΑ,ΠΕΡΙΦΕΡΕΙΑΚΗ ΕΝΟΤΗΤΑ,ΔΗΜΟΣ,ΔΗΜΟΤΙΚΗ ΕΝΟΤΗΤΑ,ΘΕΣΗ,ΙΣΧΥΣ (MW),ΤΕΧΝΟΛΟΓΙΑ,file
0,B-00279,2001-02-19,ΔΕΗ ΑΝΑΝΕΩΣΙΜΕΣ ΑΕ,ΑΔ-00488,2003-03-04,2028-03-04 00:00:00,ΒΟΡΕΙΟΥ ΑΙΓΑΙΟΥ,ΛΕΣΒΟΥ,ΛΕΣΒΟΥ,ΠΕΤΡΑΣ,ΣΤΥΨΗ/ΠΕΡΙΟΧΗ ΑΡΓΕΝΟΥ,8.0,ΓΕΩΘΕΡΜΙΑ,../../data/special_projects/202012.xlsx
0,B-00279,2001-02-19,ΔΕΗ ΑΝΑΝΕΩΣΙΜΕΣ ΑΕ,ΑΔ-00488,2003-03-04,2028-03-04 00:00:00,ΒΟΡΕΙΟΥ ΑΙΓΑΙΟΥ,ΛΕΣΒΟΥ,ΛΕΣΒΟΥ,ΠΕΤΡΑΣ,ΣΤΥΨΗ/ΠΕΡΙΟΧΗ ΑΡΓΕΝΟΥ,8.0,ΓΕΩΘΕΡΜΙΑ,../../data/special_projects/202012.xlsx
0,B-00279,2001-02-19,ΔΕΗ ΑΝΑΝΕΩΣΙΜΕΣ ΑΕ,ΑΔ-00488,2003-03-04,2028-03-04 00:00:00,ΒΟΡΕΙΟΥ ΑΙΓΑΙΟΥ,ΛΕΣΒΟΥ,ΛΕΣΒΟΥ,ΠΕΤΡΑΣ,ΣΤΥΨΗ/ΠΕΡΙΟΧΗ ΑΡΓΕΝΟΥ,8.0,ΓΕΩΘΕΡΜΙΑ,../../data/special_projects/202103.xlsx
0,B-00279,2001-02-19,ΔΕΗ ΑΝΑΝΕΩΣΙΜΕΣ ΑΕ,ΑΔ-00488,2003-03-04,2028-03-04 00:00:00,ΒΟΡΕΙΟΥ ΑΙΓΑΙΟΥ,ΛΕΣΒΟΥ,ΛΕΣΒΟΥ,ΠΕΤΡΑΣ,ΣΤΥΨΗ/ΠΕΡΙΟΧΗ ΑΡΓΕΝΟΥ,8.0,ΓΕΩΘΕΡΜΙΑ,../../data/special_projects/202012.xlsx
0,B-00279,2001-02-19,ΔΕΗ ΑΝΑΝΕΩΣΙΜΕΣ ΑΕ,ΑΔ-00488,2003-03-04,2028-03-04 00:00:00,ΒΟΡΕΙΟΥ ΑΙΓΑΙΟΥ,ΛΕΣΒΟΥ,ΛΕΣΒΟΥ,ΠΕΤΡΑΣ,ΣΤΥΨΗ/ΠΕΡΙΟΧΗ ΑΡΓΕΝΟΥ,8.0,ΓΕΩΘΕΡΜΙΑ,../../data/special_projects/202103.xlsx
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,B-00279,2001-02-19,ΔΕΗ ΑΝΑΝΕΩΣΙΜΕΣ ΑΕ,ΑΔ-00488,2003-03-04,2028-03-04 00:00:00,ΒΟΡΕΙΟΥ ΑΙΓΑΙΟΥ,ΛΕΣΒΟΥ,ΛΕΣΒΟΥ,ΠΕΤΡΑΣ,ΣΤΥΨΗ/ΠΕΡΙΟΧΗ ΑΡΓΕΝΟΥ,8.0,ΓΕΩΘΕΡΜΙΑ,../../data/special_projects/202103.xlsx
0,B-00279,2001-02-19,ΔΕΗ ΑΝΑΝΕΩΣΙΜΕΣ ΑΕ,ΑΔ-00488,2003-03-04,2028-03-04 00:00:00,ΒΟΡΕΙΟΥ ΑΙΓΑΙΟΥ,ΛΕΣΒΟΥ,ΛΕΣΒΟΥ,ΠΕΤΡΑΣ,ΣΤΥΨΗ/ΠΕΡΙΟΧΗ ΑΡΓΕΝΟΥ,8.0,ΓΕΩΘΕΡΜΙΑ,../../data/special_projects/202105.xlsx
0,B-00279,2001-02-19,ΔΕΗ ΑΝΑΝΕΩΣΙΜΕΣ ΑΕ,ΑΔ-00488,2003-03-04,2028-03-04 00:00:00,ΒΟΡΕΙΟΥ ΑΙΓΑΙΟΥ,ΛΕΣΒΟΥ,ΛΕΣΒΟΥ,ΠΕΤΡΑΣ,ΣΤΥΨΗ/ΠΕΡΙΟΧΗ ΑΡΓΕΝΟΥ,8.0,ΓΕΩΘΕΡΜΙΑ,../../data/special_projects/202108.xlsx
0,B-00279,2001-02-19,ΓΕΩΘΕΡΜΙΚΟΣ ΣΤΟΧΟΣ ΔΥΟ ΜΟΝΟΠΡΟΣΩΠΗ Α.Ε.,ΑΔ-00488,2003-03-04,2028-03-04 00:00:00,ΒΟΡΕΙΟΥ ΑΙΓΑΙΟΥ,ΛΕΣΒΟΥ,ΛΕΣΒΟΥ,ΠΕΤΡΑΣ,ΣΤΥΨΗ/ΠΕΡΙΟΧΗ ΑΡΓΕΝΟΥ,8.0,ΓΕΩΘΕΡΜΙΑ,../../data/special_projects/202110.xlsx


In [22]:
df_merged_test.drop_duplicates()

,ΑΙΤΗΣΗ,ΗΜΕΡΟΜΗΝΙΑ ΥΠΟΒΟΛΗΣ ΑΙΤΗΣΗΣ,ΕΤΑΙΡΕΙΑ,ΑΡ. ΜΗΤΡΩΟΥ ΑΔΕΙΩΝ ΡΑΕ,ΗΜΕΡΟΜΗΝΙΑ ΕΚΔ. ΑΔ.ΠΑΡΑΓΩΓΗΣ,ΗΜΕΡΟΜΗΝΙΑ ΛΗΞΗΣ ΑΔ.ΠΑΡΑΓΩΓΗΣ,ΠΕΡΙΦΕΡΕΙΑ,ΠΕΡΙΦΕΡΕΙΑΚΗ ΕΝΟΤΗΤΑ,ΔΗΜΟΣ,ΔΗΜΟΤΙΚΗ ΕΝΟΤΗΤΑ,ΘΕΣΗ,ΙΣΧΥΣ (MW),ΤΕΧΝΟΛΟΓΙΑ,file
0,B-00279,2001-02-19,ΔΕΗ ΑΝΑΝΕΩΣΙΜΕΣ ΑΕ,ΑΔ-00488,2003-03-04,2028-03-04 00:00:00,ΒΟΡΕΙΟΥ ΑΙΓΑΙΟΥ,ΛΕΣΒΟΥ,ΛΕΣΒΟΥ,ΠΕΤΡΑΣ,ΣΤΥΨΗ/ΠΕΡΙΟΧΗ ΑΡΓΕΝΟΥ,8.000,ΓΕΩΘΕΡΜΙΑ,../../data/special_projects/202012.xlsx
1,Γ-00798,2004-11-18,ΤΕΡΝΑ ΕΝΕΡΓΕΙΑΚΗ ΑΙ ΓΙΩΡΓΗΣ ΑΕ,ΑΔ-00911,2006-03-23,2021-03-23 00:00:00,ΑΤΤΙΚΗΣ,ΝΗΣΩΝ,ΥΔΡΑΣ,ΥΔΡΑΣ,ΝΗΣΙΔΑ ΑΓΙΟΣ ΓΕΩΡΓΙΟΣ,45.000,ΑΙΟΛΙΚΑ,../../data/special_projects/202012.xlsx
2,Γ-01329,2006-01-24,ΑΙΟΛΙΚΑ ΠΑΡΚΑ ΠΑΛΙΟΠΥΡΓΟΣ ΑΕ,ΑΔ-00981,2006-10-02,2021-10-02 00:00:00,ΣΤΕΡΕΑΣ ΕΛΛΑΔΟΣ,ΕΥΒΟΙΑΣ,ΚΑΡΥΣΤΟΥ,ΚΑΡΥΣΤΟΥ & ΚΑΦΗΡΕΩΣ,ΠΑΛΙΟΠΥΡΓΟΣ,13.800,ΑΙΟΛΙΚΑ,../../data/special_projects/202012.xlsx
3,Γ-01089,2005-07-22,ΤΕΡΝΑ ΕΝΕΡΓΕΙΑΚΗ ΑΙ ΓΙΩΡΓΗΣ ΑΕ,ΑΔ-00989,2006-10-16,2021-10-16 00:00:00,ΑΤΤΙΚΗΣ,ΝΗΣΩΝ,ΥΔΡΑΣ,ΥΔΡΑΣ,ΝΗΣΙΔΑ ΑΓΙΟΣ ΓΕΩΡΓΙΟΣ,24.000,ΑΙΟΛΙΚΑ,../../data/special_projects/202012.xlsx
4,Γ-01328,2006-01-24,ΑΙΟΛΙΚΑ ΠΑΡΚΑ ΠΛΑΤΑΝΟΣ ΜΟΝΟΠΡΟΣΩΠΗ ΑΕ,ΑΔ-01257,2009-06-16,2034-06-16 00:00:00,ΣΤΕΡΕΑΣ ΕΛΛΑΔΟΣ,ΕΥΒΟΙΑΣ,ΚΑΡΥΣΤΟΥ,ΜΑΡΜΑΡΙΟΥ,ΠΛΑΤΑΝΟΣ,13.800,ΑΙΟΛΙΚΑ,../../data/special_projects/202012.xlsx
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
193,Γ-011373_Α,2020-12-16,ΑΙΟΛΙΚΗ ΘΕΟΔΩΡΩΝ ΑΝΩΝΥΜΗ ΒΙΟΜΗΧΑΝΙΚΗ ΚΑΙ ΕΝΕΡ...,ΑΔ-04888,2022-11-04,2047-11-04 00:00:00,ΠΕΛΟΠΟΝΝΗΣΟΥ,ΑΡΚΑΔΙΑΣ,"ΜΕΓΑΛΟΠΟΛΗΣ,ΓΟΡΤΥΝΙΑΣ,ΤΡΙΠΟΛΗΣ","ΜΕΓΑΛΟΠΟΛΗΣ,ΤΡΙΚΟΛΩΝΩΝ,ΦΑΛΑΝΘΟΥ","ΜΠΛΕΣΙΒΟΣ, ΨΑΡΟΒΟΥΝΙ, ΛΙΜΝΕΣ ΡΟΔΙΑΣ, ΔΙΑΚΟΠΙ, ...",187.110,ΑΙΟΛΙΚΑ,../../data/special_projects/202403.xlsx
194,Γ-011380_Α,2020-12-16,ΑΙΟΛΙΚΗ ΘΕΟΔΩΡΩΝ ΑΝΩΝΥΜΗ ΒΙΟΜΗΧΑΝΙΚΗ ΚΑΙ ΕΝΕΡ...,ΑΔ-04889,2022-11-04,2047-11-04 00:00:00,"ΔΥΤΙΚΗΣ ΕΛΛΑΔΑΣ,ΠΕΛΟΠΟΝΝΗΣΟΥ","ΗΛΕΙΑΣ,ΑΡΚΑΔΙΑΣ,ΜΕΣΣΗΝΙΑΣ","ΑΝΔΡΙΤΣΑΙΝΑΣ - ΚΡΕΣΤΕΝΩΝ,ΜΕΓΑΛΟΠΟΛΗΣ,ΟΙΧΑΛΙΑΣ,...","ΑΛΙΦΕΙΡΑΣ,ΑΝΔΡΙΤΣΑΙΝΗΣ,ΓΟΡΤΥΝΟΣ,ΕΙΡΑΣ,ΖΑΧΑΡΩΣ,...","ΓΥΜΝΟΡΡΑΧΗ, ΡΟΣΚΙΟ, ΜΠΡΙΝΙΑ, ΜΠΑΝΤΑΒΑΣ, ΚΟΥΚΟΥ...",417.600,ΑΙΟΛΙΚΑ,../../data/special_projects/202403.xlsx
195,Γ-011383_Α,2020-12-16,ΑΙΟΛΙΚΗ ΘΕΟΔΩΡΩΝ ΑΝΩΝΥΜΗ ΒΙΟΜΗΧΑΝΙΚΗ ΚΑΙ ΕΝΕΡ...,ΑΔ-04890,2022-11-04,2047-11-04 00:00:00,"ΠΕΛΟΠΟΝΝΗΣΟΥ,ΔΥΤΙΚΗΣ ΕΛΛΑΔΑΣ","ΑΡΚΑΔΙΑΣ,ΑΧΑΪΑΣ,ΚΟΡΙΝΘΙΑΣ","ΓΟΡΤΥΝΙΑΣ,ΚΑΛΑΒΡΥΤΩΝ,ΤΡΙΠΟΛΗΣ,ΕΡΥΜΑΝΘΟΥ,ΣΙΚΥΩΝΙΩΝ","ΒΥΤΙΝΑΣ,ΚΑΛΑΒΡΥΤΩΝ,ΚΛΕΙΤΟΡΙΑΣ, ΚΛΕΙΤΟΡΟΣ,ΚΟΝ...","ΛΑΚΩΜΑΤΑ, ΠΑΝΑΓΙΑ, ΣΑΙΤΑΣ, ΔΡΑΚΟΒΟΥΝΙ, ΒΥΘΟΥΛΑ...",280.665,ΑΙΟΛΙΚΑ,../../data/special_projects/202403.xlsx
196,Γ-013272,2021-06-09,VOLTON ΕΛΛΗΝΙΚΗ ΕΝΕΡΓΕΙΑΚΗ ΑΝΩΝΥΜΗ ΕΤΑΙΡΕΙΑ,ΑΔ-04891,2022-11-04,2047-11-04 00:00:00,ΠΕΛΟΠΟΝΝΗΣΟΥ,"ΑΡΓΟΛΙΔΑΣ, ΑΡΚΑΔΙΑΣ","ΑΡΓΟΥΣ -ΜΥΚΗΝΩΝ, ΤΡΙΠΟΛΗΣ, ΒΟΡΕΙΑΣ ΚΥΝΟΥΡΙΑΣ, ...","ΑΡΓΟΥΣ, ΒΑΛΤΕΤΣΙΟΥ, ΒΟΡΕΙΑΣ ΚΥΝΟΥΡΙΑΣ, ΛΥΡΚΕΙΑ...",ΑΓΙΟΣ ΠΕΤΡΟΣ-ΒΑΛΤΕΤΣΙ-ΝΕΣΤΑΝΗ-ΤΣΕΜΠΕΡΟΥ,298.200,ΑΙΟΛΙΚΑ,../../data/special_projects/202403.xlsx


In [23]:
df_merged_test[df_merged_test.duplicated()]

,ΑΙΤΗΣΗ,ΗΜΕΡΟΜΗΝΙΑ ΥΠΟΒΟΛΗΣ ΑΙΤΗΣΗΣ,ΕΤΑΙΡΕΙΑ,ΑΡ. ΜΗΤΡΩΟΥ ΑΔΕΙΩΝ ΡΑΕ,ΗΜΕΡΟΜΗΝΙΑ ΕΚΔ. ΑΔ.ΠΑΡΑΓΩΓΗΣ,ΗΜΕΡΟΜΗΝΙΑ ΛΗΞΗΣ ΑΔ.ΠΑΡΑΓΩΓΗΣ,ΠΕΡΙΦΕΡΕΙΑ,ΠΕΡΙΦΕΡΕΙΑΚΗ ΕΝΟΤΗΤΑ,ΔΗΜΟΣ,ΔΗΜΟΤΙΚΗ ΕΝΟΤΗΤΑ,ΘΕΣΗ,ΙΣΧΥΣ (MW),ΤΕΧΝΟΛΟΓΙΑ,file
0,B-00279,2001-02-19 00:00:00.000,ΔΕΗ ΑΝΑΝΕΩΣΙΜΕΣ ΑΕ,ΑΔ-00488,2003-03-04 00:00:00,2028-03-04 00:00:00,ΒΟΡΕΙΟΥ ΑΙΓΑΙΟΥ,ΛΕΣΒΟΥ,ΛΕΣΒΟΥ,ΠΕΤΡΑΣ,ΣΤΥΨΗ/ΠΕΡΙΟΧΗ ΑΡΓΕΝΟΥ,8.000,ΓΕΩΘΕΡΜΙΑ,../../data/special_projects/202012.xlsx
1,Γ-00798,2004-11-18 00:00:00.000,ΤΕΡΝΑ ΕΝΕΡΓΕΙΑΚΗ ΑΙ ΓΙΩΡΓΗΣ ΑΕ,ΑΔ-00911,2006-03-23 00:00:00,2021-03-23 00:00:00,ΑΤΤΙΚΗΣ,ΝΗΣΩΝ,ΥΔΡΑΣ,ΥΔΡΑΣ,ΝΗΣΙΔΑ ΑΓΙΟΣ ΓΕΩΡΓΙΟΣ,45.000,ΑΙΟΛΙΚΑ,../../data/special_projects/202012.xlsx
2,Γ-01329,2006-01-24 00:00:00.000,ΑΙΟΛΙΚΑ ΠΑΡΚΑ ΠΑΛΙΟΠΥΡΓΟΣ ΑΕ,ΑΔ-00981,2006-10-02 00:00:00,2021-10-02 00:00:00,ΣΤΕΡΕΑΣ ΕΛΛΑΔΟΣ,ΕΥΒΟΙΑΣ,ΚΑΡΥΣΤΟΥ,ΚΑΡΥΣΤΟΥ & ΚΑΦΗΡΕΩΣ,ΠΑΛΙΟΠΥΡΓΟΣ,13.800,ΑΙΟΛΙΚΑ,../../data/special_projects/202012.xlsx
3,Γ-01089,2005-07-22 00:00:00.000,ΤΕΡΝΑ ΕΝΕΡΓΕΙΑΚΗ ΑΙ ΓΙΩΡΓΗΣ ΑΕ,ΑΔ-00989,2006-10-16 00:00:00,2021-10-16 00:00:00,ΑΤΤΙΚΗΣ,ΝΗΣΩΝ,ΥΔΡΑΣ,ΥΔΡΑΣ,ΝΗΣΙΔΑ ΑΓΙΟΣ ΓΕΩΡΓΙΟΣ,24.000,ΑΙΟΛΙΚΑ,../../data/special_projects/202012.xlsx
4,Γ-01328,2006-01-24 00:00:00.000,ΑΙΟΛΙΚΑ ΠΑΡΚΑ ΠΛΑΤΑΝΟΣ ΜΟΝΟΠΡΟΣΩΠΗ ΑΕ,ΑΔ-01257,2009-06-16 00:00:00,2034-06-16 00:00:00,ΣΤΕΡΕΑΣ ΕΛΛΑΔΟΣ,ΕΥΒΟΙΑΣ,ΚΑΡΥΣΤΟΥ,ΜΑΡΜΑΡΙΟΥ,ΠΛΑΤΑΝΟΣ,13.800,ΑΙΟΛΙΚΑ,../../data/special_projects/202012.xlsx
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,Γ-011383_Α,2020-12-16 00:00:00.000,ΑΙΟΛΙΚΗ ΘΕΟΔΩΡΩΝ ΑΝΩΝΥΜΗ ΒΙΟΜΗΧΑΝΙΚΗ ΚΑΙ ΕΝΕΡ...,ΑΔ-04890,2022-11-04 00:00:00,2047-11-04 00:00:00,"ΠΕΛΟΠΟΝΝΗΣΟΥ,ΔΥΤΙΚΗΣ ΕΛΛΑΔΑΣ","ΑΡΚΑΔΙΑΣ,ΑΧΑΪΑΣ,ΚΟΡΙΝΘΙΑΣ","ΓΟΡΤΥΝΙΑΣ,ΚΑΛΑΒΡΥΤΩΝ,ΤΡΙΠΟΛΗΣ,ΕΡΥΜΑΝΘΟΥ,ΣΙΚΥΩΝΙΩΝ","ΒΥΤΙΝΑΣ,ΚΑΛΑΒΡΥΤΩΝ,ΚΛΕΙΤΟΡΙΑΣ, ΚΛΕΙΤΟΡΟΣ,ΚΟΝ...","ΛΑΚΩΜΑΤΑ, ΠΑΝΑΓΙΑ, ΣΑΙΤΑΣ, ΔΡΑΚΟΒΟΥΝΙ, ΒΥΘΟΥΛΑ...",280.665,ΑΙΟΛΙΚΑ,../../data/special_projects/202402.xlsx
196,Γ-013272,2021-06-09 00:00:00.000,VOLTON ΕΛΛΗΝΙΚΗ ΕΝΕΡΓΕΙΑΚΗ ΑΝΩΝΥΜΗ ΕΤΑΙΡΕΙΑ,ΑΔ-04891,2022-11-04 00:00:00,2047-11-04 00:00:00,ΠΕΛΟΠΟΝΝΗΣΟΥ,"ΑΡΓΟΛΙΔΑΣ, ΑΡΚΑΔΙΑΣ","ΑΡΓΟΥΣ -ΜΥΚΗΝΩΝ, ΤΡΙΠΟΛΗΣ, ΒΟΡΕΙΑΣ ΚΥΝΟΥΡΙΑΣ, ...","ΑΡΓΟΥΣ, ΒΑΛΤΕΤΣΙΟΥ, ΒΟΡΕΙΑΣ ΚΥΝΟΥΡΙΑΣ, ΛΥΡΚΕΙΑ...",ΑΓΙΟΣ ΠΕΤΡΟΣ-ΒΑΛΤΕΤΣΙ-ΝΕΣΤΑΝΗ-ΤΣΕΜΠΕΡΟΥ,298.200,ΑΙΟΛΙΚΑ,../../data/special_projects/202402.xlsx
197,Γ-012780,2021-06-02 09:34:54.000,ΤΕΡΝΑ ΕΝΕΡΓΕΙΑΚΗ ΑΝΩΝΥΜΗ ΒΙΟΜΗΧΑΝΙΚΗ ΕΜΠΟΡΙΚΗ ...,ΑΔ-010527,2023-01-17 00:00:00,2048-01-17 00:00:00,ΣΤΕΡΕΑΣ ΕΛΛΑΔΑΣ,ΕΥΒΟΙΑΣ,ΚΑΡΥΣΤΟΥ,"ΚΑΡΥΣΤΟΥ,ΜΑΡΜΑΡΙΟΥ",ΛΟΓΚΑΚΙ - ΛΕΚΑΝΗ - ΞΕΡΟΠΟΥΣΙ - ΠΟΛΥΡΕΜΑ - ΛΥΚΟ...,79.800,ΑΙΟΛΙΚΑ,../../data/special_projects/202402.xlsx
198,Γ-013664,2022-02-01 10:24:03.000,ΤΕΡΝΑ ΕΝΕΡΓΕΙΑΚΗ ΑΝΩΝΥΜΗ ΒΙΟΜΗΧΑΝΙΚΗ ΕΜΠΟΡΙΚΗ ...,ΑΔ-010621,2023-04-05 00:00:00,2048-04-05 00:00:00,ΣΤΕΡΕΑΣ ΕΛΛΑΔΑΣ,ΕΥΒΟΙΑΣ,ΚΥΜΗΣ - ΑΛΙΒΕΡΙΟΥ,ΑΥΛΩΝΟΣ,ΜΑΚΡΟΜΙΧΑΛΗ,37.800,ΑΙΟΛΙΚΑ,../../data/special_projects/202402.xlsx


In [24]:
df_merged_test.isna().sum()

ΑΙΤΗΣΗ                             0
ΗΜΕΡΟΜΗΝΙΑ ΥΠΟΒΟΛΗΣ ΑΙΤΗΣΗΣ      283
ΕΤΑΙΡΕΙΑ                         283
ΑΡ. ΜΗΤΡΩΟΥ ΑΔΕΙΩΝ ΡΑΕ           283
ΗΜΕΡΟΜΗΝΙΑ ΕΚΔ. ΑΔ.ΠΑΡΑΓΩΓΗΣ     283
ΗΜΕΡΟΜΗΝΙΑ ΛΗΞΗΣ ΑΔ.ΠΑΡΑΓΩΓΗΣ    283
ΠΕΡΙΦΕΡΕΙΑ                       283
ΠΕΡΙΦΕΡΕΙΑΚΗ ΕΝΟΤΗΤΑ             283
ΔΗΜΟΣ                            283
ΔΗΜΟΤΙΚΗ ΕΝΟΤΗΤΑ                 283
ΘΕΣΗ                             283
ΙΣΧΥΣ (MW)                       283
ΤΕΧΝΟΛΟΓΙΑ                       283
file                               0
dtype: int64

In [25]:
merged_df.isna().sum()

ΑΙΤΗΣΗ                             0
ΗΜΕΡΟΜΗΝΙΑ ΥΠΟΒΟΛΗΣ ΑΙΤΗΣΗΣ      283
ΕΤΑΙΡΕΙΑ                         283
ΑΡ. ΜΗΤΡΩΟΥ ΑΔΕΙΩΝ ΡΑΕ           283
ΗΜΕΡΟΜΗΝΙΑ ΕΚΔ. ΑΔ.ΠΑΡΑΓΩΓΗΣ     283
ΗΜΕΡΟΜΗΝΙΑ ΛΗΞΗΣ ΑΔ.ΠΑΡΑΓΩΓΗΣ    283
ΠΕΡΙΦΕΡΕΙΑ                       283
ΠΕΡΙΦΕΡΕΙΑΚΗ ΕΝΟΤΗΤΑ             283
ΔΗΜΟΣ                            283
ΔΗΜΟΤΙΚΗ ΕΝΟΤΗΤΑ                 283
ΘΕΣΗ                             283
ΙΣΧΥΣ (MW)                       283
ΤΕΧΝΟΛΟΓΙΑ                       283
file                               0
dtype: int64

### Licenses Fetching

In [26]:
# Define the directory containing Excel files
directory = "../../data/licenses/"

# Get all Excel files in the directory
file_paths = [os.path.join(directory, file) for file in os.listdir(directory) if file.endswith(".xlsx")]
column_patterns = ['ΑΙΤΗΣΗ', 'ΜΗΤΡΩΟ', 'ΗΜΕΡΟΜΗΝΙΑ ΥΠΟΒΟΛΗΣ.*', 'ΕΤΑΙΡΕΙΑ', 'ΗΜΕΡ.*ΕΚΔ.*ΠΑΡΑΓ.*', 'ΗΜΕΡ.*ΛΗΞΗ.*ΠΑΡΑΓ.*', 'ΠΕΡΙΦΕΡΕΙΑ', 'ΠΕΡΙΦΕΡΕΙΑΚΗ ΕΝΟΤΗΤΑ', 'ΔΗΜΟ.* ', 'ΔΗΜΟΤΙΚΗ ΕΝΟΤΗΤΑ', 'ΘΕΣΗ', '.*(MW)', 'ΤΕΧΝΟΛ.*'] 

# Sort file paths based on their names to process them in order
file_paths.sort()


In [27]:
licenses_df = merge_dataframes_with_previous_files(file_paths, column_patterns)


Loading file : ../../data/licenses/Vevaioseis-APE.xlsx
Number of sheets detected: 1
Number of columns detected in 0 : 14

Loading file : ../../data/licenses/ΜΗΤΡΩΟ-ΑΔΕΙΩΝ-ΕΡΓΑ-ΑΠΕ-OKTΩΒΡΙΟΣ-2023.xlsx
Number of sheets detected: 1
Number of columns detected in 0 : 14

Loading file : ../../data/licenses/ΜΗΤΡΩΟ-ΑΔΕΙΩΝ-ΕΡΓΑ-ΑΠΕ-ΙΑΝΟΥΑΡΙΟΣ-2023.xlsx


/tmp/ipykernel_5569/310484181.py:49: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_sheet['file'] = file_path


Number of sheets detected: 1
Number of columns detected in 0 : 14

Loading file : ../../data/licenses/ΜΗΤΡΩΟ-ΑΔΕΙΩΝ-ΕΡΓΑ-ΑΠΕ-ΙΟΥΛΙΟΣ-2023.xlsx


/tmp/ipykernel_5569/310484181.py:49: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_sheet['file'] = file_path


Number of sheets detected: 1
Number of columns detected in 0 : 14

Loading file : ../../data/licenses/ΜΗΤΡΩΟ-ΑΔΕΙΩΝ-ΕΡΓΑ-ΑΠΕ-ΙΟΥΝΙΟΣ-2023.xlsx


...

/tmp/ipykernel_5569/310484181.py:49: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_sheet['file'] = file_path


Number of sheets detected: 1
Number of columns detected in 0 : 14

Loading file : ../../data/licenses/Ν.4685-ΜΗΤΡΩΟ-ΑΔΕΙΩΝ-ΕΡΓΑ-ΑΠΕ-ΣΕΠΤΕΜΒΡΙΟΣ-2022.xlsx


/tmp/ipykernel_5569/310484181.py:49: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_sheet['file'] = file_path


Number of sheets detected: 1
Number of columns detected in 0 : 14

Loading file : ../../data/licenses/Ν.4685_1.xlsx
Number of sheets detected: 1
Number of columns detected in 0 : 14

Loading file : ../../data/licenses/ΠΙΝΑΚΑΣ-ΑΙΤΙΟΛΟΓΗΜΕΝΩΝ-ΕΝΣΤΑΣΕΩΝ_ΑΝΑΡΤΗΣΗ_ΤΕΛΙΚΟ.xlsx
Number of sheets detected: 1
Number of columns detected in 0 : 9


/tmp/ipykernel_5569/310484181.py:49: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_sheet['file'] = file_path


In [28]:
licenses_df

,ΑΙΤΗΣΗ,ΗΜΕΡΟΜΗΝΙΑ ΥΠΟΒΟΛΗΣ ΑΙΤΗΣΗΣ,ΕΤΑΙΡΕΙΑ,ΑΡ. ΜΗΤΡΩΟΥ ΑΔΕΙΩΝ ΡΑΕ,ΗΜΕΡΟΜΗΝΙΑ ΕΚΔ. ΑΔ.ΠΑΡΑΓΩΓΗΣ,ΗΜΕΡΟΜΗΝΙΑ ΛΗΞΗΣ ΑΔ.ΠΑΡΑΓΩΓΗΣ,ΠΕΡΙΦΕΡΕΙΑ,ΠΕΡΙΦΕΡΕΙΑΚΗ ΕΝΟΤΗΤΑ,ΔΗΜΟΣ,ΔΗΜΟΤΙΚΗ ΕΝΟΤΗΤΑ,ΘΕΣΗ,ΜΕΓΙΣΤΗ ΙΣΧΥΣ (MW),ΤΕΧΝΟΛΟΓΙΑ,file,ΗΜΕΡ/ΝΙΑ ΕΚΔ. ΑΔ. ΠΑΡΑΓΩΓΗΣ
0,514,2001-02-19 00:00:00,ΛΑΚΜΟΣ ΕΝΕΡΓΕΙΑΚΗ ΑΕ,ΑΔ-00001,2001-05-29 00:00:00,2026-05-29 00:00:00,ΗΠΕΙΡΟΥ,ΙΩΑΝΝΙΝΩΝ,ΜΕΤΣΟΒΟΥ,ΕΓΝΑΤΙΑΣ,ΓΚΟΥΡΑ ΜΙΚΡΟ ΠΕΡΙΣΤΕΡΙ,2.155,ΜΥΗΕ,../../data/licenses/Vevaioseis-APE.xlsx,NaT
1,67,2001-02-13 00:00:00,ΑΡΑΜΠΑΤΖΗΣ Β.Γ. ΑΕ,ΑΔ-00002,2001-05-29 00:00:00,2026-05-29 00:00:00,ΚΕΝΤΡΙΚΗΣ ΜΑΚΕΔΟΝΙΑΣ,ΠΕΛΛΑΣ,ΑΛΜΩΠΙΑΣ,ΑΡΙΔΑΙΑΣ,ΤΟΥΠΛΙΤΣΑ ΠΡΟΜΑΧΩΝΑ,0.830,ΜΥΗΕ,../../data/licenses/Vevaioseis-APE.xlsx,NaT
2,38,2001-02-08 00:00:00,ΚΑΣΤΡΙ ΕΥΒΟΙΑΣ ΑΕ,ΑΔ-00022,2001-06-18 00:00:00,2041-03-22 00:00:00,ΣΤΕΡΕΑΣ ΕΛΛΑΔΟΣ,ΕΥΒΟΙΑΣ,ΚΑΡΥΣΤΟΥ,ΚΑΡΥΣΤΟΥ,ΚΑΣΤΡΙ,5.000,ΑΙΟΛΙΚΑ,../../data/licenses/Vevaioseis-APE.xlsx,NaT
3,439,2001-02-19 00:00:00,ΥΔΡΟΗΛΕΚΤΡΙΚΗ ΕΥΡΥΤΑΝΙΑΣ ΑΕ,ΑΔ-00023,2001-06-18 00:00:00,2026-06-18 00:00:00,ΣΤΕΡΕΑΣ ΕΛΛΑΔΟΣ,ΕΥΡΥΤΑΝΙΑΣ,ΑΓΡΑΦΩΝ,ΑΓΡΑΦΩΝ,ΜΟΝΑΣΤΗΡΑΚΙ,3.000,ΜΥΗΕ,../../data/licenses/Vevaioseis-APE.xlsx,NaT
4,65,2001-02-12 00:00:00,ΠΟΛΥΠΟΤΑΜΟΣ ΑΙΟΛΙΚΗ ΕΝΕΡΓΕΙΑ ΑΕ,ΑΔ-00024,2001-06-18 00:00:00,2026-06-18 00:00:00,ΣΤΕΡΕΑΣ ΕΛΛΑΔΟΣ,ΕΥΒΟΙΑΣ,ΚΑΡΥΣΤΟΥ,ΣΤΥΡΕΩΝ,ΓΚΕΡΚΙ-ΠΥΡΓΑΡΙ ΠΟΛΥΠΟΤΑΜΟΥ,12.000,ΑΙΟΛΙΚΑ,../../data/licenses/Vevaioseis-APE.xlsx,NaT
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
56,NaN,NaN,GREEN VELOCITY 2,NaN,NaN,NaN,ΘΕΣΣΑΛΙΑΣ,ΤΡΙΚΑΛΩΝ,NaN,"ΑΙΘΗΚΩΝ,ΑΣΠΡΟΠΟΤΑΜΟΥ",ΝΕΡΑΙΔΑ,NaN,ΑΙΟΛΙΚΑ,NaN,2020-09-26
57,NaN,NaN,GREEN VELOCITY 2,NaN,NaN,NaN,ΘΕΣΣΑΛΙΑΣ,ΤΡΙΚΑΛΩΝ,NaN,"ΑΣΠΡΟΠΟΤΑΜΟΥ,ΚΛΕΙΝΟΒΟΥ",ΚΑΛΤΣΑ,NaN,ΑΙΟΛΙΚΑ,NaN,2020-09-26
58,NaN,NaN,WIND ENERGY FACILITY ΜΟΝΟΠΡΟΣΩΠΗ ΙΔΙΩΤΙΚΗ ΚΕΦΑ...,NaN,NaN,NaN,ΑΝ ΜΑΚΕΔΟΝΙΑΣ ΘΡΑΚΗΣ,ΡΟΔΟΠΗΣ,NaN,ΦΙΛΛΥΡΑΣ,ΚΑΤΩ ΒΡΑΧΟΣ,NaN,ΑΙΟΛΙΚΑ,NaN,2020-09-28
59,NaN,NaN,WIND ENERGY FACILITY ΜΟΝΟΠΡΟΣΩΠΗ ΙΔΙΩΤΙΚΗ ΚΕΦΑ...,NaN,NaN,NaN,Δ ΜΑΚΕΔΟΝΙΑΣ,ΚΟΖΑΝΗΣ,NaN,ΒΛΑΣΤΗΣ,ΜΠΕΚΡΕΒΙΝΙΚΟΣ,NaN,ΑΙΟΛΙΚΑ,NaN,2020-09-28
